<a href="https://colab.research.google.com/github/neurologic/Neurophysiology-Lab/blob/main/modules/human-bci-music/Data-Explorer_EMG-basics.ipynb" target="_blank" rel="noopener noreferrer"><img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg"/></a>   

# Data Explorer

<a id="setup"></a>
# Setup

[toc](#toc)

Import and define functions

In [ ]:
#@title {display-mode: "form" }

#@markdown Run this code cell to import packages and define functions 
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.signal import hilbert,medfilt,resample, find_peaks, unit_impulse
import seaborn as sns
from datetime import datetime,timezone,timedelta
pal = sns.color_palette(n_colors=15)
pal = pal.as_hex()
import matplotlib.pyplot as plt
import random
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from pathlib import Path

from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
from ipywidgets import widgets, interact, interactive, interactive_output
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

print('Task completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))


Mount Google Drive

In [ ]:
#@title {display-mode: "form" }

#@markdown Run this cell to mount your Google Drive.

from google.colab import drive
drive.mount('/content/drive')

print('Task completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

## Import data 

Import data digitized with *Nidaq USB6211* and recorded using *Bonsai-rx* as a *.bin* file

If you would like sample this Data Explorer, but do not have data, you can download the following examples (two channels digitized at 40000). Channel 0 is the signal measured from N3 and Channel 1 is the signal measured from the Superficial Flexor muscle. 


In [ ]:
#@title {display-mode: "form" }

#@markdown Specify the file path 
#@markdown to your recorded data on Drive (find the filepath in the colab file manager:

filepath = "full filepath goes here"  #@param 
# filepath = '/Volumes/Untitled/BIOL247/data/crayfish-synaptic-connectivity/KP_20221113/spont-simult_A4_2_good-2input.bin' 

#@markdown Specify the sampling rate and number of channels recorded.

sampling_rate = None #@param
number_channels = None #@param

# downsample = False #@param
# newfs = 10000 #@param

#@markdown After you have filled out all form fields, 
#@markdown run this code cell to load the data. 

filepath = Path(filepath)

# No need to edit below this line
#################################
data = np.fromfile(Path(filepath), dtype = np.float64)
data = data.reshape(-1,number_channels)
data_dur = np.shape(data)[0]/sampling_rate
print('duration of recording was %0.2f seconds' %data_dur)

fs = sampling_rate
# if downsample:
   ## newfs = 10000 #downsample emg data
    # chunksize = int(sampling_rate/newfs)
    # data = data[0::chunksize,:]
    # fs = int(np.shape(data)[0]/data_dur)

time = np.linspace(0,data_dur,np.shape(data)[0])

print('Data upload completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

## Visualize Raw Data

Use this visualization tool if desired to get a sanity check that the data you thought you imported is actually the data that got imported.

In [ ]:
#@title {display-mode: "form"}

#@markdown Run this code cell to plot imported data. <br> 
#@markdown Use the range slider to scroll through the data in time.
#@markdown Use the channel slider to choose which channel to plot
#@markdown Be patient with the range refresh... the more data you are plotting the slower it will be. 

slider_xrange = widgets.FloatRangeSlider(
    min=0,
    max=data_dur,
    value=(0,data_dur),
    step= 0.5,
    readout=True,
    continuous_update=False,
    description='Time Range (s)')
slider_xrange.layout.width = '600px'

slider_chan = widgets.IntSlider(
    min=0,
    max=number_channels-1,
    value=0,
    step= 1,
    continuous_update=False,
    description='channel')
slider_chan.layout.width = '300px'

# a function that will modify the xaxis range
def update_plot(x,chan):
    fig, ax = plt.subplots(figsize=(10,5),num=1); #specify figure number so that it does not keep creating new ones
    starti = int(x[0]*fs)
    stopi = int(x[1]*fs)
    ax.plot(time[starti:stopi], data[starti:stopi,chan])

w = interact(update_plot, x=slider_xrange, chan=slider_chan);

For a more extensive ***RAW*** Data Explorer than the one provided in the above figure, use the [DataExplorer.py](https://raw.githubusercontent.com/neurologic/Neurophysiology-Lab/main/howto/Data-Explorer.py) application found in the [howto section](https://neurologic.github.io/Neurophysiology-Lab/howto/Dash-Data-Explorer.html) of the course website.

<a id="one"></a>
# I. Movement Coding and Recruitment

Python has built-in algorithms for detecting "peaks" in a signal. However, it will detect *all* peaks. Therefore, the function takes in arguments that specify parameters for minimum height that can count as a peak and a minimum acceptible interval between independent peaks. 

First, we will detect all the peaks in the signal within a set of thresholds. This will give the time of each peak.  


## Detect motor unit events

In [ ]:
#@title {display-mode: "form"}

#@markdown Indicate which channel has the EMG signal (should be 0).

spike_channel = 0 #@param

#@markdown Then, run the code cell to create an interactive plot with a slider to scroll 
#@markdown through the raw data and set an upper and lower peak detection threshold.
#@markdown You can set the polarity of the peak detection: upward (1) or downward (-1) peaks. 
#@markdown Peak times (according to your threshold) will be plotted using red markers. <br>
#@markdown Only the signal within the previously specified bout times will be processed (shaded gray regions in the plot).

slider_xrange = widgets.FloatRangeSlider(
    min=0,
    max=data_dur,
    value=(0,data_dur),
    step=0.01,
    readout_format='.2f',
    continuous_update=False,
    readout=True,
    description='xrange (s)'
)
slider_xrange.layout.width = '600px'

slider_yrange = widgets.FloatRangeSlider(
    min=np.min(data[:,spike_channel])-0.1,
    max=np.max(data[:,spike_channel])+0.1,
    value=[np.min(data[:,spike_channel])-0.1,np.max(data[:,spike_channel])+0.1],
    step=0.01,
    readout_format='.2f',
    continuous_update=False,
    readout=True,
    description='yrange'
)
slider_yrange.layout.width = '600px'



slider_threshold_low = widgets.FloatSlider(
    min=0,
    max=np.max([np.max(data[:,spike_channel]),np.abs(np.min(data[:,spike_channel]))])+0.1,
    value=0,
    step=0.001,
    readout_format='.3f',
    continuous_update=False,
    readout=True,
    description='lower threshold')
slider_threshold_low.layout.width = '600px'

slider_threshold_high = widgets.FloatSlider(
    min=0,
    max=np.max([np.max(data[:,spike_channel]),np.abs(np.min(data[:,spike_channel]))])+0.1,
    value=np.max([np.max(data[:,spike_channel]),np.abs(np.min(data[:,spike_channel]))])+0.1,
    step=0.001,
    readout_format='.3f',
    continuous_update=False,
    readout=True,
    description='upper threshold')
slider_threshold_high.layout.width = '600px'

radio_polarity = widgets.RadioButtons(
    options=[1, -1],
    value=1,
    description='peaks polarity',
    disabled=False
)

iei_text = widgets.Text(
    value='0.005',
    placeholder='0.005',
    description='min IEI (seconds)',
    style = {'description_width': '200px'},
    disabled=False
)


# a function that will modify the xaxis range
def update_plot(xrange,yrange,thresh_low_,thresh_high_,polarity,iei):
    fig, ax = plt.subplots(figsize=(10,6),num=1); #specify figure number so that it does not keep creating new ones
    fig.tight_layout()    
    
    win_0 = int(xrange[0]*fs)
    win_1 = int(xrange[1]*fs)

    xtime = np.linspace(xrange[0],xrange[1],(win_1 - win_0))

    ax.plot(xtime,data[win_0:win_1,spike_channel],color='black',linewidth=1)
    ax.set_ylim(yrange[0],yrange[1]);
    
    ax.hlines(thresh_low_*polarity, xrange[0],xrange[1],linestyle='--',color='green',zorder=3)
    ax.hlines(thresh_high_*polarity, xrange[0],xrange[1],linestyle='--',color='orange',zorder=3)
    
    
    # calculate spike times based on threshold
    d = float(iei)*fs #minimum time allowed between distinct events
    r = find_peaks(data[:,spike_channel]*polarity,height=thresh_low_,distance=d)

    spike_times = r[0]/fs
    mask_spikes = r[1]['peak_heights']<thresh_high_
    # spike_times = spike_times[mask_spikes]

    inwin_inds = []
    for b_ in bouts_list:
        inwin_inds.extend(r[0][(spike_times>b_[0]) & (spike_times<b_[1]) & mask_spikes])
    inwin_inds = np.asarray(inwin_inds)
    inwin_inds = np.in1d(r[0],inwin_inds)
    
    df_props = pd.DataFrame({
            'height': r[1]['peak_heights'][inwin_inds]*polarity,
            'spikeT' : spike_times[inwin_inds],
            'spikeInd' : r[0][inwin_inds]
                })
    
    spike_times = spike_times[mask_spikes]
    
    inwin_spikes = spike_times[(spike_times>(xrange[0])) & (spike_times<(xrange[1]))]
    ax.scatter(inwin_spikes,[np.mean(data[:,spike_channel])] * len(inwin_spikes),
          zorder=3,color='red',s=20)

    for b_ in bouts_list:
        ax.axvspan(b_[0], b_[1], color = 'black', alpha=0.1)    


    return spike_times,df_props
    

w_spikes_ = interactive(update_plot, xrange=slider_xrange, 
             yrange=slider_yrange,
             thresh_low_=slider_threshold_low,thresh_high_=slider_threshold_high,polarity=radio_polarity,iei=iei_text);

display(w_spikes_)

In [ ]:
#@title {display-mode: "form"}

#@markdown Run this cell to finalize the list of spike times after settling on a threshold in the interactive plot. 
spike_times,df_props = w_spikes_.result

## Define Trial Times

In [ ]:
#@title {display-mode: "form"}

#@markdown For this experiment, you don't have precise trial onset markers, but you should be able to 
#@markdown estimate the onset and offset of individual trials. 
#@markdown Specify the trial range as follows: [start of trial, end of trial] <br>
#@markdown You will only be able to examine one trial at a time. 

trial_range = [0,60] #@param

#@markdown Then run this code cell to programatically define the trial time range.

## Plot processed data

In [ ]:
#@title {display-mode: "form"}

#@markdown Run this cell to create a plot of instantaneous peak rate and a plot of peak amplitude at each peak time.
spike_times,df_props = w_spikes_.result

mask = ((spike_times>bouts_list[0]) & (spike_times<bouts_list[1]))

hfig,ax = plt.subplots(figsize=(5,4))
ax.scatter(spike_times[mask],np.diff(spike_times[mask],append(spike_times[mask][-1]-spike_times[mask][-2])))
ax.set_ylabel('peak times',fontsize=14)
ax.set_xlabel('peak rate',fontsize=14)
           
hfig,ax = plt.subplots(figsize=(5,4))
ax.scatter(spike_times[mask],df_props[mask]['height'])
ax.set_ylabel('peak times',fontsize=14)
ax.set_xlabel('peak amplitude',fontsize=14)

n,bins = np.histogram(df_props['height'][mask],bins = 500) # calculate the histogram
bins = bins[1:]
hfig,ax = plt.subplots(figsize=(5,4))
ax.step(bins,n,color='black')
ax.set_ylabel('count',fontsize=14)
ax.set_xlabel('amplitude',fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14);

<a id="one"></a>
# III. Fatigue

To process the EMG signal in this experiment, we will calculate a ***moving RMS***. 


## Define Trial Times

In [ ]:
#@title {display-mode: "form"}

#@markdown For this experiment, you don't have precise trial onset markers, but you should be able to 
#@markdown estimate the onset and offset of individual trials. 
#@markdown Specify the trial range as follows: [start of trial, end of trial] <br>
#@markdown You will only be able to examine one trial at a time. 

trial_range = [0,60] #@param

#@markdown Then run this code cell to programatically define the trial time range.

In [ ]:
#@title {display-mode: "form"}

#@markdown Specify a window size (in seconds) for the moving average

window_size = 1 #@param

#@markdown Specify which channel you want to analyze

channel_ = 0 #@param

window_size = int(window_size*fs)

#@markdown Then run this code cell to plot the RMS

a = data[:,channel_]

def window_rms(a, window_size):
    a2 = np.power(a,2)
    window = np.ones(window_size)/float(window_size)
    return np.sqrt(np.convolve(a2, window, 'same'))

rms_data = window_rms(a, window_size)

xtime = np.linspace(0,data_dur,np.shape(data)[1])

hfig,ax = plt.subplots(figsize=(5,4))
ax.plot(xtime,rms_data)
ax.set_ylabel('rms',fontsize=14)
ax.set_xlabel('time (s)',fontsize=14)